In [ ]:
#Required modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pywt
import timeit
import glob
from scipy import stats
from astropy.io import fits
import os,sys
from astropy.table import Table
import time

In [ ]:
# continuously load modules (for ppxf external codes)
%load_ext autoreload
%autoreload 2

In [ ]:
#Add the amaru lib path
sys.path.append(".")

In [ ]:
#Import the amaru function
from amaru_v020 import amaru 

In [ ]:
MY_PATH='MY_PATH' #replace with your path to the directories
#Stellar templates directory
emiles=MY_PATH+'/EMILES_PADOVA00_BASE_CH_FITS/*.*'

#FeII templates directory
fe2_e=MY_PATH+'/uvofeii_templates/uvopfe_conct/uvofeii1000kms.txt'
fe2_t=[fe2_e]
#Balmer continuum and Balmer High order directory
Bal_cont=MY_PATH+'/BalCont/Bal*'
Bal_Ho=MY_PATH+'/BalCont/*M1000.dat'
Bal_t=[Bal_cont,Bal_Ho]

#For dereddening

#sdf_maps=MY_PATH+'/sdfmaps/sfddata-master/'
sdf_maps=None #You can add a sdf maps for dereddening

In [ ]:
c1='mkdir ../plots_fits_test' #Where plots are going to be saved
c2='mkdir ../deviations_data_test' #Where the deviations results are going tobe saved
c3='mkdir ../residuals_significance' #Where the residuals significance results are going to be saved
cmds=[c1,c2,c3]
for k in cmds:
    
    os.system(k)

In [ ]:
#DataFrame for comparison
times_df=pd.DataFrame(columns=['name','time_model1','time_pp1','time_dwt','time_pp2','time_compar','time_save','total_time','redshift',
                               'best_model','flag_2md_comp','DWT_sugg','DWT_comment','DWT_one_reg_sig','DWT_wo3_sign','DWT_dv_nr','ifh3','ifh4',
                              'case_bels','case_winds','comparison_sc','chi2_1','chi2_2','my_chi2'])

In [ ]:
#SDSS spectra files PATH (example)
files=glob.glob('../examples_spec_2/spec*')
len(files)

In [ ]:
start_time_t = timeit.default_timer()
# List to save files directories where a problem in the fit was encounter
not_fitted=[]
dir_plots='../plots_fits_syn_2/'
#Counter for the DataFrame
k=0
#Counter for files in loop
cnt_f=0
#for dir_t in files:
for dir_t in files:    
    cnt_f+=1
    print(f'--------file {cnt_f} of {len(files)}--------')
    #To save deviations with the spec name
    hn1=dir_t.split(sep='spec-')
    hn=hn1[-1].split(sep='.f')
    devdf_dir='../deviations_data_test/devidf-'+hn[0]+'.csv'
    try:
        start_time = timeit.default_timer()
        mm=amaru(dir_t,EMILES_Dir=emiles,sdfmaps=sdf_maps,st_func=0,fe2_temps=fe2_t,Balmer_tem=Bal_t,save_op=1,plot_model_ini=1,my_plot_dir=dir_plots,devdf_dir=devdf_dir)
        end_time = timeit.default_timer()
        time_tot=np.round(end_time-start_time)
        plt.close('all')
        
        #To save results in the DataFrame for comparison
        pp=mm.pp
        ppsol=pp.sol
        idx_belm=mm.idx_bels
        h3_f=ppsol[idx_belm][2]  #index are from the model used to fit, related with the components indexing
        h4_f=ppsol[idx_belm][3]  #first index 3 is coming from the model where the BELs are in the index 3 of the model
        #Add a row with the results
        k+=1
        times_df.loc[k]=[dir_t,mm.time_model1,mm.time_pp1,mm.time_dwt,mm.time_pp2,mm.time_compar,mm.time_save,time_tot,mm.z,mm.best_model,
                        mm.comp_re,mm.new_model,mm.comment_wavelet,mm.sig_one_reg,mm.wo3_sig,mm.dv_nr,h3_f,h4_f,mm.case_bels,mm.case_winds,
                        mm.comparison_sc,mm.chi21,mm.chi22,mm.my_chi2]

    except:
        #Save the directory with problems during the fit
        not_fitted.append(dir_t)
    
end_time_t = timeit.default_timer()
print('Wavelet finished in : '+str(np.round(end_time_t-start_time_t))+'s')

In [ ]:
not_fitted

In [ ]:
times_df